# Homework - Grover MaxCut

The places where you have enter code are marked with `# YOUR CODE HERE`.

In [1]:
import cirq
from cirq import H, X, Y, Z, CX, inverse

## Question 1 (4 points)

Write a function, `oracle010`, that implements an oracle that marks the state $|010 \rangle$. The function `oracle010` has

* input: `qq`, a 3-qubit register 
* returns: None

The function should append a sequence of gates to `qq` to mark the state $|010\rangle$ only. Don't append any measurements to `qq`.

To help you test the function, we have provided the `grover_diffusion` and `grover` functions.

In [2]:
def oracle010(qq):
    # YOUR CODE HERE
    qq_list = []
    qq_list.append(cirq.X(qq[0]))
    qq_list.append(cirq.X(qq[2]))
    qq_list.append(cirq.CZ(qq[0], qq[2]).controlled_by(qq[1]))
    qq_list.append(cirq.X(qq[0]))
    qq_list.append(cirq.X(qq[2]))
    #qq_list.append(cirq.Z(qq[1]).controlled_by(qq[0], qq[2]))

    return qq_list


In [3]:
# visualize your implemented gates
qqTest = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append(oracle010(qqTest))
circuit

0: ───X───@───X───
          │
1: ───────@───────
          │
2: ───X───@───X───

In [4]:
# To check your solution, we need some to implement grover
def grover_diffusion(qq,n):
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[n-1]).controlled_by(*(qq[0:n-1]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [5]:
def grover(trials_number):
    n=3
    qq = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*qq))  

    for i in range(2):
        circuit.append(oracle010(qq))
        circuit.append(grover_diffusion(qq,n))
    circuit.append(cirq.measure(*qq, key='result'))

    # determine the statistics of the measurements
    s = cirq.Simulator() 
    
    samples = s.run(circuit, repetitions=trials_number)

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    print(counts)
    return counts.get('010')

In [6]:
# run grover to test if your function gives the right answer
grover(100)

Counter({'010': 97, '110': 2, '001': 1})


97

In [7]:
# hidden tests in this cell will be used for grading.

## Question 2 (6 points)

Write an oracle for the graph described below to check whether it admits a valid 2-coloring. 

The function `oracle2` has

* input: `qq`, a 12-qubit register 
* returns: None

The function should append a sequence of gates to `qq` to check for a valid coloring. Don't append any measurements to `qq`.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation2` functions.



$G$ has 5 vertices. Use qubits 0-4 for the vertices, 5-10 for the edges and 11 as the ancilla.

The list of edges:
(0,3)
(0,4)
(1,3)
(1,4)
(2,3)
(2,4)


In [8]:
def oracle2(qq):
    # YOUR CODE HERE
    qq_list = []
    edges_list = [(0,3), (0,4), (1,3), (1,4), (2,3), (2,4)]

    # circuit for vqe solution
    #for i in edges_list:
    #    qq_list.append(cirq.ZZ(qq[i[0]], qq[i[1]]))
    
    def edge_check(a, b, c):
        qq_list.append(CX(qq[a], qq[c]))
        qq_list.append(CX(qq[b], qq[c]))
    store_q = 5
    for i in edges_list:
        edge_check(i[0],i[1],store_q)
        store_q+=1
    
    qq_list.append(X(qq[11]).controlled_by(*(qq[5:11])))

    
    return qq_list

In [9]:
# We need some code so you can check your solution
def oracle_computation2(qq):
    yield oracle2(qq)
    yield Z(qq[11])
    yield inverse(oracle2(qq))  

In [10]:
def grover2(trials_number):    
    import cirq
    from cirq import X, H, Z, inverse, CX
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(12)
    n=5
    
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation2(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [11]:
#You can use this cell to test your solution
shots=1000
grover2(shots)

Counter({'00011': 463,
         '11100': 437,
         '11110': 4,
         '10100': 6,
         '10110': 4,
         '01011': 7,
         '11000': 4,
         '00010': 5,
         '10111': 4,
         '00001': 1,
         '01111': 3,
         '01010': 3,
         '00100': 2,
         '10010': 5,
         '01000': 3,
         '01110': 3,
         '00000': 1,
         '11011': 3,
         '01101': 5,
         '11010': 2,
         '10101': 3,
         '10011': 3,
         '11101': 5,
         '00101': 3,
         '00111': 5,
         '11111': 2,
         '10000': 4,
         '01100': 4,
         '00110': 1,
         '10001': 1,
         '01001': 2,
         '11001': 2})

In [12]:
# hidden tests in this cell will be used for grading.

## Question 3 (10 points)

Write an oracle for the graph described below to check whether there exists a coloring with 4 edges connecting vertices with different colors.

The function `oracle3` has

* input: `qq`, a 13-qubit register 
* returns: None

The function should append a sequence of gates to `qq` to check whether there exists a coloring with 4 edges connecting vertices with different colors. Don't append any measurements to `qq`.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation3` functions.


$G$ has 4 vertices. Use qubits 0-3 for the vertices, 4-8 for the edges, 9-11 for the addition and 12 as the ancilla.

The list of edges: 
(0,1)
(0,2)
(0,3)
(1,2)
(1,3)



In [13]:
def oracle3(qq):
    # YOUR CODE HERE
    # in the graph you have 5 edges, so the maximum number of edges that you have to count will be 5, 
    # which is 101 in binary -> therefore you need 3 bits to represent the sum (the bit 9, 10, 11). 
    # and then check if this sum is equal to 4
    
    qq_list = []
    edges_list = [(0,1),(0,2),(0,3),(1,2),(1,3)]
    
    def edge_check(a, b, c):
        qq_list.append(CX(qq[a], qq[c]))
        qq_list.append(CX(qq[b], qq[c]))
    store_q = 4
    for i in edges_list:
        edge_check(i[0],i[1],store_q)
        store_q+=1

    # addition
    node=4
    n = node+5

    # add qubit 4
    qq_list.append(cirq.CX(qq[node], qq[n]))
    
    # add qubits 5-6
    for j in range(5,7):
        qq_list.append(cirq.CCX(qq[j], qq[n], qq[n+1]))
        qq_list.append(cirq.CX(qq[j], qq[n]))
    
    # add qubits 7-8
    for j in range(7,9):
        qq_list.append(cirq.X(qq[n+2]).controlled_by(qq[j], qq[n],qq[n+1]))
        qq_list.append(cirq.CCX(qq[j], qq[n], qq[n+1]))
        qq_list.append(cirq.CX(qq[j], qq[n]))
    
    qq_list.append(X(qq[12]).controlled_by(*(qq[9:12]),control_values=[0,0,1]))
    return qq_list

In [14]:
# We need some code so you can check your solution
def oracle_computation3(qq):
    yield oracle3(qq)
    yield Z(qq[12])
    yield inverse(oracle3(qq))  

In [15]:
import cirq
from cirq import X, H, Z, inverse, CX, CCX
    
def grover3(trials_number):    
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(13)
    n=4

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation3(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [16]:
#You can use this cell to test your solution
shots=1000
grover3(shots)

Counter({'0011': 482,
         '1100': 449,
         '0001': 6,
         '1110': 8,
         '1000': 3,
         '0100': 11,
         '1101': 6,
         '1010': 2,
         '1001': 4,
         '1011': 3,
         '0010': 3,
         '0111': 4,
         '0000': 8,
         '1111': 4,
         '0101': 2,
         '0110': 5})

In [17]:
# hidden tests in this cell will be used for grading.